In [1]:
!pip install simpletransformers pandas torch scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00

In [2]:
import pandas as pd
import torch
from simpletransformers.classification import ClassificationModel, ClassificationArgs
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

Using device: cuda


In [14]:
train_df = pd.read_csv("/content/twitter_training.csv")
test_df = pd.read_csv("/content/twitter_test.csv")

In [15]:
print("\nTraining Data Preview:")
display(train_df.head())


Training Data Preview:


,Tweet ID,Entity,sentiment,Tweet Content
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [16]:
print("\nTest Data Preview:")
display(test_df.head())


Test Data Preview:


,Tweet ID,Entity,sentiment,Tweet Content
0,3364,Facebook,Irrelevant,I mentioned on Facebook that I was struggling ...
1,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
2,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
3,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
4,4433,Google,Neutral,Now the President is slapping Americans in the...


In [17]:
train_df = train_df[['Tweet Content', 'sentiment']]
test_df = test_df[['Tweet Content', 'sentiment']]

In [18]:
train_df.dropna(inplace=True)
test_df.dropna(inplace=True)
train_df.columns = train_df.columns.str.strip()
test_df.columns = test_df.columns.str.strip()
train_df["sentiment"] = train_df["sentiment"].astype(str).str.strip()
test_df["sentiment"] = test_df["sentiment"].astype(str).str.strip()

In [19]:
label_map = {"Negative": 0, "Neutral": 1, "Positive": 2}
train_df["sentiment"] = train_df["sentiment"].map(label_map)
test_df["sentiment"] = test_df["sentiment"].map(label_map)

In [20]:
train_df.dropna(inplace=True)
test_df.dropna(inplace=True)

In [21]:
train_df["sentiment"] = train_df["sentiment"].astype(int)
test_df["sentiment"] = test_df["sentiment"].astype(int)

In [22]:
print("\nTraining Dataset After Preprocessing:")
display(train_df.head())


Training Dataset After Preprocessing:


,Tweet Content,sentiment
0,im getting on borderlands and i will murder yo...,2
1,I am coming to the borders and I will kill you...,2
2,im getting on borderlands and i will kill you ...,2
3,im coming on borderlands and i will murder you...,2
4,im getting on borderlands 2 and i will murder ...,2


In [23]:
print("\nValidation Dataset After Preprocessing:")
display(test_df.head())


Validation Dataset After Preprocessing:


,Tweet Content,sentiment
1,BBC News - Amazon boss Jeff Bezos rejects clai...,1
2,@Microsoft Why do I pay for WORD when it funct...,0
3,"CSGO matchmaking is so full of closet hacking,...",0
4,Now the President is slapping Americans in the...,1
5,Hi @EAHelp I’ve had Madeleine McCann in my cel...,0


In [24]:
model_args = ClassificationArgs()
model_args.num_train_epochs = 2
model_args.train_batch_size = 16
model_args.eval_batch_size = 16
model_args.learning_rate = 2e-5
model_args.overwrite_output_dir = True
model_args.save_best_model = True
model_args.evaluate_during_training = True
model_args.save_eval_checkpoints = False
model_args.save_model_every_epoch = False
model_args.output_dir = "xlm-roberta-sentiment-model"
model_args.best_model_dir = "xlm-roberta-best-model"

In [25]:
model = ClassificationModel(
    "xlmroberta", "xlm-roberta-base",
    num_labels=3,
    args=model_args,
    use_cuda=torch.cuda.is_available()
)

print("\n👌 Model Loaded Successfully!")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]


👌 Model Loaded Successfully!


/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:484: UserWarning: use_multiprocessing automatically disabled as xlmroberta fails when using multiprocessing for feature conversion.
  warnings.warn(


In [26]:
train_df, valid_df = train_test_split(train_df, train_size=0.8, shuffle=True, random_state=42)

In [27]:
print("\n🚀 Training Started...")
model.train_model(train_df, eval_df=valid_df)


🚀 Training Started...


/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:610: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:882: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = amp.GradScaler()


Running Epoch 1 of 2:   0%|          | 0/3056 [00:00<?, ?it/s]

...

  0%|          | 0/24 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:1505: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():
/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:905: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():
/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:1453: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/24 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:1505: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


(6112,
 defaultdict(list,
             {'global_step': [2000, 3056, 4000, 6000, 6112],
              'train_loss': [0.7991390228271484,
               0.5850505828857422,
               0.8860912322998047,
               0.7606229782104492,
               0.2953801155090332],
              'mcc': [0.6463354446258339,
               0.7036023220906545,
               0.7290789113191096,
               0.785922294591625,
               0.7846397735374697],
              'eval_loss': [0.5991534737979665,
               0.5036864268234352,
               0.4995242075203291,
               0.3986190993801441,
               0.39732155394709967]}))

In [28]:
print("\n🔍 Evaluating Model...")
result, model_outputs, wrong_predictions = model.eval_model(valid_df, acc=accuracy_score)


🔍 Evaluating Model...


/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:1453: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/24 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/765 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:1505: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


In [29]:
print(f"\nModel Accuracy: {result['acc']:.4f}")


Model Accuracy: 0.8569


In [30]:
predictions, raw_outputs = model.predict(valid_df["Tweet Content"].tolist())

  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/765 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/simpletransformers/classification/classification_model.py:2188: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast():


In [31]:
print("\n📊 Classification Report:\n")
print(classification_report(valid_df["sentiment"], predictions))


📊 Classification Report:

              precision    recall  f1-score   support

           0       0.88      0.88      0.88      4427
           1       0.83      0.82      0.83      3678
           2       0.85      0.86      0.86      4120

    accuracy                           0.86     12225
   macro avg       0.86      0.85      0.86     12225
weighted avg       0.86      0.86      0.86     12225

